Notebook to parse the 52 week planer.
Currently working according to the cell positions.

In [25]:
# pandas for data manipulation
# requests for posting the data
import pandas as pd
import requests

Reading files and sheets

In [26]:
# creating the excel file
file = pd.ExcelFile("../excel_files/Summit 52 week PPM Schedule-2019.xlsx")

In [27]:
#making list of dataframe
#reading from column 8 onwards, since privious were non-informative
list_of_df = []
for i in range(len(file.sheet_names)):
    list_of_df.append(pd.read_excel(file,sheet_name = file.sheet_names[i],header =[2]).T.loc[:,8:])

Cleaning the data 

In [28]:
for each_df in list_of_df:
    print(each_df.shape)

(67, 116)
(58, 156)


In [29]:
# working with the first dataframe 
first = list_of_df[1]

In [30]:
# dropping all the rows and columns which are completly
# rows
for each_df in list_of_df:
    each_df.dropna(axis =0,how ='all',inplace = True)
# columns
    each_df.dropna(axis =1,how ='all',inplace = True)

In [31]:
for each_df in list_of_df:
    print(each_df.shape)

(58, 114)
(58, 153)


In [32]:
# adding index to the dataframes
index =[]
for _ in range(1,59):
    index.append(_)

In [33]:
for each_df in list_of_df:
    each_df.index = index

In [34]:
# replacing the tick with 1
for each_df in list_of_df:
    each_df.replace(to_replace='√', value=1, inplace=True)

In [35]:
# creating a list of list of list
# 1. list -number of elements =number of sheets
# 2. list - for each sheet
# 3. list - each week
dates = []
for each_df in list_of_df:
    date =[]
    for each_column in (each_df):
        index = each_df[each_column].index[each_df[each_column] ==1]
        date.append(index)
    dates.append(date)

In [36]:
list_of_df[1].sample(10)

,8,9,10,11,12,13,14,15,16,17,...,154,155,156,157,158,159,160,161,162,163
1,Daily Visual Inspection,Lighting,Water Tank Levels,Water Quality,Chillers,Exhaust Fans,Fresh Air Fans,Pumps,Lifts,UPS,...,Lift machine room UPS (car emergency lights),Lifts,Passenger lifts L1 to L8,Service lifts S1 & S2,Parking lifts P1 to P3,MLCP lifts M1 & M2,EARTH PITS,ALL EARTH PITS,ENERGY METER CALIBRATION,Dual source meter's
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1KVA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,AMC,AMC,AMC,AMC,NaN,NaN,NaN,NaN
5,Daily,Daily,Daily,Daily,Daily,Daily,Daily,Daily,Daily,Daily,...,Quarterly,NaN,Monthly,Monthly,Monthly,Monthly,NaN,YEARLY,NaN,YEARLY
6,1,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Converting to the workorder dataframe

In [37]:
# creating a list of dataframes
# similar to a list of all the data from each sheet
df_workorder = []
for i in range(len(list_of_df)):
    title = list(list_of_df[i].iloc[0,:])
    rating = list(list_of_df[i].iloc[1,:])
    location = list(list_of_df[i].iloc[2,:])
    amc = list(list_of_df[i].iloc[3,:])
    frequency = list(list_of_df[i].iloc[4,:])
    date = dates[i]
    
    workorder = {"Title":title,
            "Rating":rating,
            "Location":location,
            "AMC/Non AMC":amc,
            "Frequency":frequency,
            "Week Number":date}
    df_workorder.append(pd.DataFrame(workorder))
    

In [17]:
# to export the complete data into json files
'''
for i in range(len(list_of_df)):
    df_workorder[i].to_json(path_or_buf ="../json_data/"+file.sheet_names[i]+".json" ,orient = 'records')
'''

In [ ]:
service_url = "http://ec2-13-233-84-234.ap-south-1.compute.amazonaws.com:5000"
#service_url = "http://127.0.0.1:5000"
sessionToken = "0"
workOrderId = 0

curlpostcmd = "curl -i -X POST "

init_file = 0

In [27]:
def workorder_create_df(df,i):
        robj = 'workorder'
        url = service_url + '/api/' + robj
        data = df.to_dict('records')[i]
        global sessionToken
        header = {
            "sessionToken": sessionToken
        }

        curlcmd = curlpostcmd + " -H " + "\'" + 'sessionToken:' + header['sessionToken'] + "\'"  +  " -d " + "\'" + json.dumps(data) + "\'" +  " " + url
        log_curlcmd(curlcmd, "Create workorder,having Title,Rating,Location,AMC,Frequency,Week Number")
        response = requests.post(url, json = data, headers = header)
        assert(response.status_code == 201)
        global workOrderId
        workOrderId = response.json()["id"]

In [ ]:
for each_df in df_workorder:
    for i in range(len(each_df):
                   workorder_create_df(each_df,i)

In [18]:
print("Done")

Done
